In [1]:
# pip3 install opencv-python numpy matplotlib
import cv2 as cv

import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import os
from scipy.signal import find_peaks
direc= os.getcwd()


In [19]:
# Create necessary dirs if they do not exist
direc= os.getcwd()
dirs = [direc+ "/linesdetected", direc+ "/output", direc+ "/cropped"]

for path in dirs:
    isExist = os.path.exists(path)

    if not isExist:
  
      # Create a new directory because it does not exist 
      os.makedirs(path)
      print("The new directory -{0} \n is created!".format(path))

# Algorithm 1: Histogram projection

In [21]:
import numpy as np
import cv2 as cv
def fixLineBeginningAndEnds(inputImage, orig):
    
    BlackFrame = np.zeros(inputImage.shape, np.uint8)
    contours, _ = cv.findContours(
        inputImage, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        x, y, w, h = cv.boundingRect(contour)
        #omit short contours and omit lines due to scanned documents edges
        if (h <=3 or y <55 or y>1295):
            continue
        cropped = inputImage[y:y + h, x:x + w]
        orig1 =cv.bitwise_not(orig[y:y + h, x:x + w].copy())
        space = np.linspace(1, w, 15)
        average = sum(np.sum(orig1,axis=1)/15)
        start = 0
        stop = 0
        previous_val = 0
        for count,k in enumerate(space):
            val = sum(np.sum(orig1[:,:int(k)],axis=1))
            if(val <average/3):
                start = start+1
            if(val < (14.5 * average)):
                stop = stop+1
            #print(start, stop)
        
        cv.rectangle(BlackFrame,(x + int(start* w/15),y),(x+w-int(w*(15-stop)/15),y+h),(255, 255, 255),-1)
    return BlackFrame

In [22]:
from skimage.io import imread
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
from skimage.filters import sobel
import numpy as np
from PIL import Image
import glob
import os
import cv2 as cv

def horizontal_projections(img):
    return np.sum(img, axis=1)

def calculateXProjection(array):
    return np.sum(array,axis=0).tolist()

def find_peak_regions(hpp, divider=2.5):
    threshold = (np.max(hpp)-np.min(hpp))/divider
    peaks = []
    peaks_index = []
    for i, hppv in enumerate(hpp):
        if hppv < threshold:
            peaks.append([i, hppv])
    return peaks

def main():
    direc= os.getcwd()

    for img in glob.glob(direc + '/images' + '/*.jpg'):
        try:
            print("Image: \"" + img + "\" is being processed...")
            
            image = cv.imread(img)
            image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
            th, image = cv.threshold(image, 170, 255, cv.THRESH_BINARY)
            rightLeftEdge= int(600/5)
            rightRightEdge=int(770/5)
            sobel_image = sobel(image)
            hpp = calculateXProjection(sobel_image)

            temp = [sum(hpp[i:i+5]) for i in range(0, len(hpp), 5)]


            leftParagraphCornerIndex = int(temp[40:60].index(min(temp[40:60]))+40)*5 
            RightParagraphCornerIndex = int(temp[rightLeftEdge:rightRightEdge].index(min(temp[rightLeftEdge:rightRightEdge])) +
                                           rightLeftEdge) *5 

            sobel_image_left = sobel(image[:, :(leftParagraphCornerIndex)])
            sobel_image_middle = sobel(image[:, leftParagraphCornerIndex:RightParagraphCornerIndex])
            sobel_image_right = sobel(image[:, RightParagraphCornerIndex:])

            #print(sobel_image_left.shape,sobel_image_middle.shape, sobel_image_right.shape)
            imag = [sobel_image_left, sobel_image_middle, sobel_image_right]

            new_image=[]
            for i in range(3):
                hpp = horizontal_projections(imag[i])
                peaks = find_peak_regions(hpp)
                if(len(peaks) == 0):
                    new_image.append(np.zeros(imag[i].shape, np.uint8))
                    continue
                    
                peaks_index = np.array(peaks)[:,0].astype(int)

                segmented_img = np.ones(imag[i].shape, np.uint8)

                r,c = segmented_img.shape
                for ri in range(r):
                    if ri in peaks_index:
                        segmented_img[ri, :] = 0
                
                new_image.append(segmented_img)
    
            sections_of_image=[]
            for k in new_image:
                height, width, *rest= k.shape
                for i in range(height):
                    for j in range(width):
                        if(j<5):
                            if (k[i, j] ==1):
                                k[i, j] =0
                sections_of_image.append(k)

            predictions = np.concatenate((sections_of_image[0], sections_of_image[1], sections_of_image[2]), axis=1)
            
            
            
            predictions[predictions == 1] = 255
            output = fixLineBeginningAndEnds(predictions,image)
            im = Image.fromarray(output)
            
            output_dir = "output/" + img[img.find("images")+7:-4] + ".png"
            im.save(output_dir)
            print("processed file is saved: " + output_dir)
            
        except Exception as e:
            print(e)

if __name__ == "__main__":
    main()  

Image: "C:\Users\datog\Downloads\for-upload\DIA-Project/images\fmb-cb-55-018r.jpg" is being processed...
processed file is saved: output/fmb-cb-55-018r.png
Image: "C:\Users\datog\Downloads\for-upload\DIA-Project/images\fmb-cb-55-018v.jpg" is being processed...
processed file is saved: output/fmb-cb-55-018v.png
Image: "C:\Users\datog\Downloads\for-upload\DIA-Project/images\fmb-cb-55-019r.jpg" is being processed...
processed file is saved: output/fmb-cb-55-019r.png
Image: "C:\Users\datog\Downloads\for-upload\DIA-Project/images\fmb-cb-55-019v.jpg" is being processed...
processed file is saved: output/fmb-cb-55-019v.png
Image: "C:\Users\datog\Downloads\for-upload\DIA-Project/images\fmb-cb-55-020r.jpg" is being processed...
processed file is saved: output/fmb-cb-55-020r.png
Image: "C:\Users\datog\Downloads\for-upload\DIA-Project/images\fmb-cb-55-021r.jpg" is being processed...
processed file is saved: output/fmb-cb-55-021r.png
Image: "C:\Users\datog\Downloads\for-upload\DIA-Project/images\f

processed file is saved: output/fmb-cb-55-045r.png
Image: "C:\Users\datog\Downloads\for-upload\DIA-Project/images\fmb-cb-55-045v.jpg" is being processed...
processed file is saved: output/fmb-cb-55-045v.png
Image: "C:\Users\datog\Downloads\for-upload\DIA-Project/images\fmb-cb-55-046r.jpg" is being processed...
processed file is saved: output/fmb-cb-55-046r.png
Image: "C:\Users\datog\Downloads\for-upload\DIA-Project/images\fmb-cb-55-046v.jpg" is being processed...
processed file is saved: output/fmb-cb-55-046v.png
Image: "C:\Users\datog\Downloads\for-upload\DIA-Project/images\fmb-cb-55-047r.jpg" is being processed...
processed file is saved: output/fmb-cb-55-047r.png
Image: "C:\Users\datog\Downloads\for-upload\DIA-Project/images\fmb-cb-55-047v.jpg" is being processed...
processed file is saved: output/fmb-cb-55-047v.png
Image: "C:\Users\datog\Downloads\for-upload\DIA-Project/images\fmb-cb-55-048r.jpg" is being processed...
processed file is saved: output/fmb-cb-55-048r.png
Image: "C:\Us

processed file is saved: output/fmb-cb-55-071v.png
Image: "C:\Users\datog\Downloads\for-upload\DIA-Project/images\fmb-cb-55-072r.jpg" is being processed...
processed file is saved: output/fmb-cb-55-072r.png
Image: "C:\Users\datog\Downloads\for-upload\DIA-Project/images\fmb-cb-55-072v.jpg" is being processed...
processed file is saved: output/fmb-cb-55-072v.png
Image: "C:\Users\datog\Downloads\for-upload\DIA-Project/images\fmb-cb-55-073r.jpg" is being processed...
processed file is saved: output/fmb-cb-55-073r.png
Image: "C:\Users\datog\Downloads\for-upload\DIA-Project/images\fmb-cb-55-073v.jpg" is being processed...
processed file is saved: output/fmb-cb-55-073v.png
Image: "C:\Users\datog\Downloads\for-upload\DIA-Project/images\fmb-cb-55-074r.jpg" is being processed...
processed file is saved: output/fmb-cb-55-074r.png
Image: "C:\Users\datog\Downloads\for-upload\DIA-Project/images\fmb-cb-55-074v.jpg" is being processed...
processed file is saved: output/fmb-cb-55-074v.png
Image: "C:\Us

Please, check predicted lines in the folder "output".

In [20]:
#Second Method

# Algorithm 2: Naive Hough Transform

In [ ]:
import cv2
import numpy as np
import cv2
import numpy as np
from math import atan2
import os
import glob
from PIL import Image
# pip3 install opencv-python numpy matplotlib
import cv2 as cv

import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import os
from scipy.signal import find_peaks
direc= os.getcwd()
direc= os.getcwd()
i =0
for img in glob.glob(direc + '/images' + '/*.jpg'):
    image = cv2.imread(img)
#cv2.imshow('orig',image)
#cv2.waitKey(0)

#grayscale
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    #cv2.imshow('binary',gray)
    cv2.waitKey(0)
    
    blurred_image = cv2.GaussianBlur(gray, (9, 9), 0)
    #cv2.imshow('blurred',blurred_image)
#binary
    ret,thresh = cv2.threshold(blurred_image,180,255,cv2.THRESH_BINARY_INV)
    #cv2.imshow('binary',thresh)
    cv2.waitKey(0)

#dilation
    kernel = np.ones((3,10), np.uint8)
    img_dilation = cv2.dilate(thresh, kernel, iterations=1)
    #cv2.imshow('dilated',img_dilation)

    kernel = np.ones((8,20), np.uint8)
    img_erosion = cv2.erode(img_dilation, kernel, iterations=1)
    #cv2.imshow('second',img_erosion)
    
    kernel = np.ones((1,30), np.uint8)  # note this is a horizontal kernel
    img_dilation = cv2.dilate(img_erosion, kernel, iterations=1)
    kernel = np.ones((3,10), np.uint8)
    img_erosion = cv2.erode(img_dilation, kernel, iterations=1)
    #cv2.imshow('second',img_dilation)

#im = Image.fromarray(img_erosion)
#im.save("your_file.jpeg")

    BlackFrame = np.zeros(image.shape, np.uint8)

#gray = cv2.cvtColor(e_im, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(img_dilation, 75, 150)
    cv2.imshow('second',edges)
    lines = cv2.HoughLinesP(edges, 1, np.pi/90, 30, minLineLength = 40, maxLineGap=50)
    for line in lines:
        x1, y1, x2, y2 = line[0]
    
        if(x1!=x2 and (abs((y1-y2)/(x1-x2)) < np.pi/90)):
            cv2.line(BlackFrame, (x1, y1), (x2, y2), (255,255,255), 8)
    im = Image.fromarray(BlackFrame)
    im.save("output/" + img[img.find("images")+7:])
    i=i+1
#cv2.imshow("linesEdges", edges)
#cv2.imshow("linesDetected", BlackFrame)
#cv2.waitKey(0)
#cv2.destroyAllWindows()